[![Fixel Algorithms](https://fixelalgorithms.co/images/CCExt.png)](https://fixelalgorithms.gitlab.io)

# Machine Learning Methods

## Supervised Learning - Regression - Polynomial Fit

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 0.1.000 | 09/02/2023 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/MachineLearningMethods/2023_01/0022RegressorPolynomialFit.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Machine Learning

# Miscellaneous
import os
from platform import python_version
import random

# Typing
from typing import Tuple

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Jupyter
from IPython import get_ipython
from IPython.display import Image, display
from ipywidgets import Dropdown, FloatSlider, interact, IntSlider, Layout

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

In [ ]:
# Configuration
%matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())

In [ ]:
# Constants

FIG_SIZE_DEF    = (8, 8)
ELM_SIZE_DEF    = 50
CLASS_COLOR     = ('b', 'r')
EDGE_COLOR      = 'k'
MARKER_SIZE_DEF = 10
LINE_WIDTH_DEF  = 2


In [ ]:
# Fixel Algorithms Packages


In [ ]:
# Parameters

# Data Generation
numSamples  = 30
noiseStd    = 0.3

vP = np.array([0.5, 2, 5])
polynomDeg = 2

# Data Visualization
gridNoiseStd = 0.05
numGridPts = 250

In [ ]:
# Auxiliary Functions

def PlotRegressionData( mX: np.ndarray, vY: np.ndarray, hA:plt.Axes = None, figSize: Tuple[int, int] = FIG_SIZE_DEF, elmSize: int = ELM_SIZE_DEF, classColor: Tuple[str, str] = CLASS_COLOR, axisTitle: str = None ) -> plt.Axes:

    if hA is None:
        hF, hA = plt.subplots(figsize = figSize)
    else:
        hF = hA.get_figure()
    
    if np.ndim(mX) == 1:
        mX = np.reshape(mX, (mX.size, 1))

    numSamples = len(vY)
    numDim     = mX.shape[1]
    if (numDim > 2):
        raise ValueError(f'The features data must have at most 2 dimensions')
    
    # Work on 1D, Add support for 2D when needed
    # See https://matplotlib.org/stable/api/toolkits/mplot3d.html
    hA.scatter(mX[:, 0], vY, s = elmSize, color = classColor[0], edgecolor = 'k', label = f'Samples')
    hA.axvline(x = 0, color = 'k')
    hA.axhline(y = 0, color = 'k')
    hA.set_xlabel('${x}_{1}$')
    # hA.axis('equal')
    if axisTitle is not None:
        hA.set_title(axisTitle)
    hA.legend()
    
    return hA

def PlotPolyFit( vX: np.ndarray, vY: np.ndarray, vP: np.ndarray = None, P: int = 1, numGridPts: int = 1001, hA:plt.Axes = None, figSize: Tuple[int, int] = FIG_SIZE_DEF, markerSize: int = MARKER_SIZE_DEF, lineWidth: int = LINE_WIDTH_DEF, axisTitle: str = None ):

    if hA is None:
        hF, hA = plt.subplots(1, 2, figsize = figSize)
    else:
        hF = hA[0].get_figure()

    numSamples = len(vY)

    # Polyfit
    vW    = np.polyfit(vX, vY, P)
    
    # MSE
    vHatY = np.polyval(vW, vX)
    MSE   = (np.linalg.norm(vY - vHatY) ** 2) / numSamples
    
    # Plot
    xx  = np.linspace(np.floor(np.min(vX)), np.ceil(np.max(vX)), numGridPts)
    yy  = np.polyval(vW, xx)

    hA[0].plot(vX, vY, '.r', ms = 10, label = '$y_i$')
    hA[0].plot(xx, yy, 'b',  lw = 2,  label = '$\hat{f}(x)$')
    hA[0].set_title (f'$P = {P}$\nMSE = {MSE}')
    hA[0].set_xlabel('$x$')
    # hA[0].axis(lAxis)
    hA[0].grid()
    hA[0].legend()
    
    hA[1].stem(vW[::-1], label = 'Estimated')
    if vP is not None:
        hA[1].stem(vP[::-1], linefmt = 'C1:', markerfmt = 'D', label = 'Ground Truth')
    numTicks = len(vW) if vP is None else max(len(vW), len(vP))
    hA[1].set_xticks(range(numTicks))
    hA[1].set_title('Coefficients')
    hA[1].set_xlabel('$w$')
    hA[1].legend()

    # return hA


## Generate / Load Data

In the following we'll generate data according to the following model:

$$ y_{i} = f \left( x_{i} \right) + \epsilon_{i} $$

Where

$$ f \left( x \right) = \frac{1}{2} x^{2} + 2x + 5 $$


In [ ]:
# Defining the Generating Function
def f( vX: np.ndarray, vP: np.ndarray ):
    # return 0.25 * (vX ** 2) + 2 * vX + 5
    return np.polyval(vP, vX)


hF = lambda vX: f(vX, vP)

In [ ]:
# Loading / Generating Data

vX = np.linspace(-2, 2, numSamples, endpoint = True) + (gridNoiseStd * np.random.randn(numSamples))
vN = noiseStd * np.random.randn(numSamples)
vY = hF(vX) + vN

print(f'The features data shape: {vX.shape}')
print(f'The labels data shape: {vY.shape}')

### Plot Data

In [ ]:
# Display the Data

PlotRegressionData(vX, vY)

plt.show()

## Train Polyfit Regressor

The PolyFit optimization problem is given by:

$$ \arg \min_{\boldsymbol{w}} {\left\| \boldsymbol{y} - \boldsymbol{\Phi} \boldsymbol{w} \right|}_{2}^{2} $$

Where

$$
\boldsymbol{\Phi} = \begin{bmatrix} 1 & x_{1} & x_{1}^{2} & \cdots & x_{1}^{p} \\
1 & x_{2} & x_{2}^{2} & \cdots & x_{2}^{p} \\
\vdots & \vdots & \vdots &  & \vdots \\
1 & x_{N} & x_{N}^{2} & \cdots & x_{N}^{p}
\end{bmatrix}
$$

This is a _polyfit_ with hyper parameter $p$.

The optimal weights are calculated by linear system solvers.  
Yet it is better to use solvers optimized for this task, such as:

 * NumPy: [`polyfit`](https://numpy.org/doc/stable/reference/generated/numpy.polyfit.html).
 * SciKit Learn: [`LinearRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) combined with [`PolynomialFeatures`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html).

In this notebook we'll use the NumPy's implementation.

* <font color='brown'>(**#**)</font> For arbitrary $\Phi$ the above becomes a _linear regression_ problem.

In [ ]:
# Polynomial Fit

# The order of Polynomial p(x) = w[0] * x**deg + ... + w[deg]
# Hence we need to show it reversed:
vW = np.polyfit(vX, vY, polynomDeg)

for ii in range(polynomDeg + 1):
    print(f'The coefficient of degree {ii}: {vW[-1 - ii]:0.3f}')


### Effect of the Polynomial Degree $p$

The degree of the polynomial is basically the DoF of the model.  
Tuning it is the way (Along with Regularization) to avoid _underfit_ and _overfit_.

In [ ]:
hPolyFit = lambda P: PlotPolyFit(vX, vY, vP = vP, P = P)
pSlider = IntSlider(min = 0, max = 31, step = 1, value = 0, layout = Layout(width = '30%'))
interact(hPolyFit, P = pSlider)
plt.show()

* <font color='red'>(**?**)</font> What happens when the degree of the polynomial is higher than the number of samples?
* <font color='red'>(**?**)</font> What would be the optimal $\boldsymbol{w}$ in case the model matrix is given by $\boldsymbol{\Phi} = \begin{bmatrix} 5 & 2 x_{1} & \frac{1}{2} x_{1}^{2} \\
5 & 2 x_{2} & \frac{1}{2} x_{2}^{2} & \\
\vdots & \vdots & \vdots \\
5 & 2 x_{N} & \frac{1}{2} x_{N}^{2}
\end{bmatrix}$?
* <font color='brown'>(**#**)</font> The properties of the model matrix are important. As we basically after the best approximation of the data in the space its columns spans. For instance, for Polynomial Fit, we ca use better basis for polynomials.

### Sensitivity to Support

We'll show the effect of the support, given a number of sample on the estimated weights (_Coefficients_).

In [ ]:
vN = 20 * noiseStd * np.random.randn(numSamples)

def GenDataByRadius( vP, P, vN, valR: float = 1.0 ):

    vX = np.linspace(-valR, valR, np.size(vN), endpoint = True)
    vY = f(vX, vP) + vN
    
    PlotPolyFit(vX, vY, vP = vP, P = P)
    


In [ ]:
hGenDataByRadius = lambda valR: GenDataByRadius(vP, polynomDeg, vN, valR)

rSlider = FloatSlider(min = 0.1, max = 50.0, step = 0.1, value = 0.1, layout = Layout(width = '30%'))
interact(hGenDataByRadius, valR = rSlider)

plt.show()

In [ ]:
# Doing the above manually 

def GenDataRadius( vP, vN, valR: float = 1.0 ):

    vX = np.linspace(-valR, valR, np.size(vN), endpoint = True)
    vY = f(vX, vP) + vN

    return vX, vY

vR = np.linspace(0.1, 50, 20)


for valR in vR:
    vX, vY = GenDataRadius(vP, 0.5 * vN, valR)
    vW = np.polyfit(vX, vY, polynomDeg)

    vYPred = np.polyval(vW, vX)
    valMSESamples = np.mean(np.square(vYPred - vY))
    valMSECoeff = np.mean(np.square(vW - vP))
    print(f'The Samples MSE     : {valMSESamples}.')
    print(f'The Coefficients MSE: {valMSECoeff}.')
    print(f'The Estimated Coef  : {vW}')